# 🧠 Tree of Thoughts con Gemma vía Ollama
Este notebook permite ejecutar pruebas de razonamiento tipo Tree-of-Thoughts (ToT) con el modelo `gemma3:1b` usando Ollama. Puedes introducir una pregunta o tema, generar ramificaciones, elegir manualmente el mejor camino y guardar el resultado como `.json`.

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import json
from datetime import datetime

# Clase base para usar gemma con memoria
class Chatbot:
    def __init__(self, base_prompt, model_name="gemma3:1b", temperature=0.8):
        self.base_prompt = base_prompt
        self.chat_prompt = ChatPromptTemplate.from_messages([
            ('system', self.base_prompt),
            MessagesPlaceholder(variable_name='memory'),
            ('human', '{input}')
        ])
        self.llm = OllamaLLM(model=model_name, temperature=temperature)
        self.memory = []
        self.pipeline = self.chat_prompt | self.llm

    def __call__(self, prompt):
        response = self.pipeline.invoke({
            'input': HumanMessage(prompt),
            'memory': self.memory
        })
        self.memory.append(HumanMessage(content=prompt))
        self.memory.append(AIMessage(content=response))
        return response.content

In [ ]:
# Configuración inicial
tema = "¿Por qué es importante el análisis matemático en ciencia de datos?"
ramas_por_paso = 3
num_pasos = 2

chatbot = Chatbot(
    base_prompt="Eres un tutor inteligente que razona paso a paso antes de responder preguntas académicas. Responde en español."
)

In [ ]:
# Ejecución del ciclo ToT
pasos = []
razonamiento_actual = tema

for paso in range(1, num_pasos+1):
    print(f"\n🧠 Paso {paso}: Generando {ramas_por_paso} pensamientos para: {razonamiento_actual}\n")
    opciones = []
    for i in range(ramas_por_paso):
        pensamiento = chatbot(f"Propón una idea para: {razonamiento_actual}")
        print(f"{i}. {pensamiento}\n")
        opciones.append(pensamiento)

    seleccion = int(input(f"Selecciona el índice del mejor pensamiento para el paso {paso}: "))
    razonamiento_actual = opciones[seleccion]
    pasos.append({"paso": paso, "opciones": opciones, "seleccion": seleccion})

In [ ]:
# Generar respuesta final a partir del razonamiento elegido
respuesta_final = chatbot(f"Redacta una respuesta completa basada en esta idea: {razonamiento_actual}")
print("\n📘 Respuesta Final:\n")
print(respuesta_final)

In [ ]:
# Guardar resultados
resultado = {
    "tema": tema,
    "pasos": pasos,
    "resultado_final": respuesta_final,
    "timestamp": datetime.now().isoformat()
}

filename = f"resultados_tot_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print(f"\n✅ Resultado guardado en {filename}")